## Peer-graded Assignment week 3, notebook 3 of 3: Segmenting and Clustering Neighborhoods in Toronto
This notebook is used for assignment week 3, Datascience Capstone.
At first, I copy relevant parts from notebook 1 and 2:

In [1]:
!pip install beautifulsoup4
!pip install lxml
!pip install requests
from bs4 import BeautifulSoup
import requests
import pandas as pd
import numpy as np

In [2]:
source=requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text
soup = BeautifulSoup(source,'lxml')
table=soup.find('table',{'class':'wikitable sortable'})
table_rows = table.find_all('tr')
res = []
for tr in table_rows:
    td = tr.find_all('td')
    row = [tr.text.strip() for tr in td if tr.text.strip()]
    if row:
        res.append(row)
df = pd.DataFrame(res, columns=["Postcode", "Borough", "Neighbourhood"])
df1=df[df.Borough != 'Not assigned']
df2=df1.groupby(["Postcode","Borough"])['Neighbourhood'].apply(','.join).reset_index()
df2['Neighbourhood'] = np.where(df2['Neighbourhood'] == 'Not assigned', df2['Borough'], df2['Neighbourhood'])
geodata = pd.read_csv("https://cocl.us/Geospatial_data") 
geodata.rename(columns={'Postal Code':'Postcode'}, inplace=True)
df3 = pd.merge(df2, geodata, on='Postcode')
df3.head(5)

,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge,Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood,Morningside,West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476


In [3]:
df3.shape

(103, 5)

In [4]:
df3.groupby('Borough').count()

,Postcode,Neighbourhood,Latitude,Longitude
Borough,,,,
Central Toronto,9,9,9,9
Downtown Toronto,18,18,18,18
East Toronto,5,5,5,5
East York,5,5,5,5
Etobicoke,12,12,12,12
Mississauga,1,1,1,1
North York,24,24,24,24
Queen's Park,1,1,1,1
Scarborough,17,17,17,17


In [5]:
# convert an address into latitude and longitude values
!pip install geopy
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

# Matplotlib and associated plotting modules
!pip install matplotlib
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

# map rendering library
!pip install folium
import folium

# library to handle JSON files
import json

# tranform JSON file into a pandas dataframe
from pandas.io.json import json_normalize 

Explore and cluster the neighborhoods in Toronto. You can decide to work with only boroughs that contain the word Toronto and then replicate the same analysis we did to the New York City data. It is up to you.

Just make sure:
1. to add enough Markdown cells to explain what you decided to do and to report any observations you make.
2. to generate maps to visualize your neighborhoods and how they cluster together.

Once you are happy with your analysis, submit a link to the new Notebook on your Github repository. (3 marks)

In [6]:
address = 'Toronto, Canada'
geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.653963, -79.387207.


In [7]:
# create map of Toronto using latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighbourhood in zip(df3['Latitude'], df3['Longitude'], df3['Borough'], df3['Neighbourhood']):
    label = '{}, {}'.format(neighbourhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

In [8]:
central_toronto = df3[df3['Borough'] == 'Central Toronto'].reset_index(drop=True)
central_toronto.head(10)

,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M4N,Central Toronto,Lawrence Park,43.728020,-79.388790
1,M4P,Central Toronto,Davisville North,43.712751,-79.390197
2,M4R,Central Toronto,North Toronto West,43.715383,-79.405678
3,M4S,Central Toronto,Davisville,43.704324,-79.388790
4,M4T,Central Toronto,"Moore Park,Summerhill East",43.689574,-79.383160
5,M4V,Central Toronto,"Deer Park,Forest Hill SE,Rathnelly,South Hill,...",43.686412,-79.400049
6,M5N,Central Toronto,Roselawn,43.711695,-79.416936
7,M5P,Central Toronto,"Forest Hill North,Forest Hill West",43.696948,-79.411307
8,M5R,Central Toronto,"The Annex,North Midtown,Yorkville",43.672710,-79.405678


In [9]:
# create map of Central Toronro using latitude and longitude values
map_central_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, label in zip(central_toronto['Latitude'], central_toronto['Longitude'], central_toronto['Neighbourhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_central_toronto)  
    
map_central_toronto

Define Foursquare Credentials and Version:

In [10]:
CLIENT_ID = 'RWLY0RHFLLB2X5L5SF2FV243QOFFSINLN0HZUOT1BFEBCK5G' # your Foursquare ID
CLIENT_SECRET = 'SBFSJDUVKAH2EJRJFRDS1ZYIQYLTKOTXAYREQN5AENT4L5PA' # your Foursquare Secret
VERSION = '20190609' # Foursquare API version

Get the top 100 venues that are in neighbourhood Davisville in Central Toronto within a radius of 500 meters.

I start with getting Davisville's latitude and longitude and convering those to scalar values which is necessary in order to use Foursquare.

In [11]:
davisville_lat=np.asscalar(df3.loc[df3['Neighbourhood'] == 'Davisville']['Latitude'].values)

In [12]:
davisville_long=np.asscalar(df3.loc[df3['Neighbourhood'] == 'Davisville']['Longitude'].values)

In [13]:
LIMIT = 100 # limit of number of venues returned by Foursquare API
radius = 500 # define radius
# create URL
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    davisville_lat, 
    davisville_long, 
    radius, 
    LIMIT)
url

'https://api.foursquare.com/v2/venues/explore?&client_id=RWLY0RHFLLB2X5L5SF2FV243QOFFSINLN0HZUOT1BFEBCK5G&client_secret=SBFSJDUVKAH2EJRJFRDS1ZYIQYLTKOTXAYREQN5AENT4L5PA&v=20190609&ll=43.7043244,-79.3887901&radius=500&limit=100'

Send the GET request and examine the results:

In [32]:
results = requests.get(url).json()
#results

Foursquare's "get_category_type" function that extracts the category of the venue:

In [33]:
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

Structure json into a pandas dataframe:

In [34]:
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head(5)

,name,categories,lat,lng
0,Downtown Toronto,Neighborhood,43.653232,-79.385296
1,Art Gallery of Ontario,Art Gallery,43.654003,-79.392922
2,Nathan Phillips Square,Plaza,43.652270,-79.383516
3,Four Seasons Centre for the Performing Arts,Concert Hall,43.650592,-79.385806
4,Pai,Thai Restaurant,43.647923,-79.388579


In [35]:
nearby_venues.shape

(100, 4)

Explore all of the neighbourhoods in Central Toronto in the similar way:

In [36]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighbourhood', 
                  'Neighbourhood Latitude', 
                  'Neighbourhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [37]:
central_toronto

,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M4N,Central Toronto,Lawrence Park,43.728020,-79.388790
1,M4P,Central Toronto,Davisville North,43.712751,-79.390197
2,M4R,Central Toronto,North Toronto West,43.715383,-79.405678
3,M4S,Central Toronto,Davisville,43.704324,-79.388790
4,M4T,Central Toronto,"Moore Park,Summerhill East",43.689574,-79.383160
5,M4V,Central Toronto,"Deer Park,Forest Hill SE,Rathnelly,South Hill,...",43.686412,-79.400049
6,M5N,Central Toronto,Roselawn,43.711695,-79.416936
7,M5P,Central Toronto,"Forest Hill North,Forest Hill West",43.696948,-79.411307
8,M5R,Central Toronto,"The Annex,North Midtown,Yorkville",43.672710,-79.405678


In [38]:
central_toronto_venues = getNearbyVenues(names=central_toronto['Neighbourhood'],
                                         latitudes=central_toronto['Latitude'],
                                         longitudes=central_toronto['Longitude'])

Lawrence Park
Davisville North
North Toronto West
Davisville
Moore Park,Summerhill East
Deer Park,Forest Hill SE,Rathnelly,South Hill,Summerhill West
Roselawn
Forest Hill North,Forest Hill West
The Annex,North Midtown,Yorkville


In [26]:
print(central_toronto_venues.shape)
central_toronto_venues.head()

(110, 7)


,Neighbourhood,Neighbourhood Latitude,Neighbourhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Lawrence Park,43.728020,-79.388790,Lawrence Park Ravine,43.726963,-79.394382,Park
1,Lawrence Park,43.728020,-79.388790,Zodiac Swim School,43.728532,-79.382860,Swim School
2,Lawrence Park,43.728020,-79.388790,TTC Bus #162 - Lawrence-Donway,43.728026,-79.382805,Bus Line
3,Davisville North,43.712751,-79.390197,Sherwood Park,43.716551,-79.387776,Park
4,Davisville North,43.712751,-79.390197,Summerhill Market North,43.715499,-79.392881,Food & Drink Shop


In [27]:
central_toronto_venues.groupby('Neighbourhood').count()

,Neighbourhood Latitude,Neighbourhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighbourhood,,,,,,
Davisville,38,38,38,38,38,38
Davisville North,8,8,8,8,8,8
"Deer Park,Forest Hill SE,Rathnelly,South Hill,Summerhill West",14,14,14,14,14,14
"Forest Hill North,Forest Hill West",4,4,4,4,4,4
Lawrence Park,3,3,3,3,3,3
"Moore Park,Summerhill East",2,2,2,2,2,2
North Toronto West,16,16,16,16,16,16
Roselawn,2,2,2,2,2,2
"The Annex,North Midtown,Yorkville",23,23,23,23,23,23


In [29]:
print('There are {} uniques categories in Central Toronto.'.format(len(central_toronto_venues['Venue Category'].unique())))

There are 61 uniques categories in Central Toronto.


## Cluster Neighbourhoods
I'll go back to analyzing the whole city of Toronro instead of one borough since it won't be so much to cluster otherwise :)
Therefore, I'll need to repeat some of the steps from above at first.

In [31]:
LIMIT = 1000 # limit of number of venues returned by Foursquare API
radius = 5000 # define radius
# create URL
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    latitude, 
    longitude, 
    radius, 
    LIMIT)
url

'https://api.foursquare.com/v2/venues/explore?&client_id=RWLY0RHFLLB2X5L5SF2FV243QOFFSINLN0HZUOT1BFEBCK5G&client_secret=SBFSJDUVKAH2EJRJFRDS1ZYIQYLTKOTXAYREQN5AENT4L5PA&v=20190609&ll=43.653963,-79.387207&radius=5000&limit=1000'

In [30]:
# set number of clusters
kclusters = 5

toronto_grouped_clustering = toronto_grouped.drop('Neighbourhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

NameError: name 'toronto_grouped' is not defined